In [12]:
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns 
sns.set()

In [7]:
#Find common search parameters excluding year and census location at:
#https://wonder.cdc.gov/controller/saved/D76/D82F692

dfne1999 = pd.read_csv('Capstone_1/1999 Cause of death Midwest.txt')
dfne1999.to_csv
dfne2008 = pd.read_csv('')
dfne2008
dfne2018 = pd.read_csv('')
dfne2018
dfmw1999 = pd.read_csv('')
dfmw1999
dfmw2008 = pd.read_csv('')
dfmw2008
dfmw2018 = pd.read_csv('')
dfmw2018
dfs1999 = pd.read_csv('')
dfs1999
dfs2008 = pd.read_csv('')
dfs2008
dfw2018 = pd.read_csv('')
dfw2018


Centers for Disease Control and Prevention, National Center for Health Statistics. Underlying Cause of Death 1999-2018 on CDC WONDER Online Database, released in 2020. Data are from the Multiple Cause of Death Files, 1999-2018, as compiled from data provided by the 57 vital statistics jurisdictions through the Vital Statistics Cooperative Program. Accessed at http://wonder.cdc.gov/ucd-icd10.html on May 1, 2020 5:00:25 PM


In [11]:

df.shape
df.head(10)
df.describe()

,Year,Year Code,Deaths
count,1060.000000,1060.000000,1060.000000
mean,2011.141509,2011.141509,41.005660
std,4.643959,4.643959,68.815268
min,2008.000000,2008.000000,10.000000
25%,2008.000000,2008.000000,13.000000
50%,2008.000000,2008.000000,20.000000
75%,2018.000000,2018.000000,40.000000
max,2018.000000,2018.000000,929.000000
